<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Corpus Linguistics - Phase 1 - Claudia

The purpose of this document is extracting data for a pilot Lexical Multi-Dimensional Analysis.

## Required Python packages

- pandas
- PyMuPDF
- tqdm

## Import the required libraries

In [1]:
import os
import sys
import pandas as pd
import fitz  # PyMuPDF
import logging
import re
from pathlib import Path
from tqdm import tqdm
import shutil
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Define input variables

In [2]:
input_directory = '3Corpus_VEm_dividido'
output_directory = 'dataset'
log_filename = f"{output_directory}/cl_st1_ph1_claudia.log"

## Create output directory

In [3]:
# Check if the output directory already exists. If it does, do nothing. If it doesn't exist, create it.
if os.path.exists(output_directory):
    print('Output directory already exists.')
else:
    try:
        os.makedirs(output_directory)
        print('Output directory successfully created.')
    except OSError as e:
        print('Failed to create the directory:', e)
        sys.exit(1)

Output directory already exists.


## Set up logging

In [4]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename = log_filename
)

## Normalise the filenames

Task: Rename PDF files in `input_directory` so that:
- Replace any hyphen between issue and part with underscore
- Replace the dot between issue and part with underscore
- Preserve zero padding and other parts. Examples:

  VEm_01.1.pdf -> VEm_01_1.pdf

  Vem-16-1.pdf -> VEm_16_1.pdf

  VEm-18-2.pdf -> VEm_18_2.pdf

  VEm-19-10.pdf -> VEm_19_10.pdf

In [5]:
pattern = re.compile(r'^ve[mn][\-_]?(\d{2})[.\-_](\d+)\.pdf$', re.IGNORECASE)

renamed = []
skipped = []

for p in Path(input_directory).glob('*.pdf'):
    m = pattern.match(p.name)
    if not m:
        skipped.append(p.name)
        continue
    issue = m.group(1)
    part = m.group(2)
    new_name = f"VEm_{issue}_{part}.pdf"
    new_path = p.with_name(new_name)
    if new_path != p:
        p.rename(new_path)
        renamed.append((p.name, new_name))

pd.DataFrame(renamed, columns=['old_name', 'new_name'])

,old_name,new_name
0,Vem-16-1.pdf,VEm_16_1.pdf
1,Vem-16-2.pdf,VEm_16_2.pdf
2,Vem-16-3.pdf,VEm_16_3.pdf
3,Vem-16-4.pdf,VEm_16_4.pdf
4,Vem-16-5.pdf,VEm_16_5.pdf
...,...,...
154,VEm_30.05.pdf,VEm_30_05.pdf
155,VEm_30.1.pdf,VEm_30_1.pdf
156,VEm_30.2.pdf,VEm_30_2.pdf
157,VEm_30.3.pdf,VEm_30_3.pdf


## Import the directory structure into a DataFrame

In [6]:
# Initialising an empty list to hold the directory information
directory_data = []

# Walking through the directory structure
for root, dirs, files in os.walk(input_directory):
    #for file in tqdm(files, desc='Processing files'):
    for file in files:
        if file.endswith('.pdf'):
            try:
                # Getting the full file path
                file_path = os.path.join(root, file)
                # Splitting the root into individual directories
                directory_parts = root.split(os.sep)
                # Creating a dictionary for this file's information
                file_info = {'File': file}
                # Adding the file path to the dictionary
                file_info['File Path'] = file_path
                # Adding each part of the directory to the dictionary with appropriate keys
                for i, part in enumerate(directory_parts):
                    file_info[f"Directory Level {i+1}"] = part

                # Opening the PDF file
                document = fitz.open(file_path)
                # Extracting text from each page
                text = ''
                for page_num in range(document.page_count):
                    page = document[page_num]
                    text += page.get_text()

                file_info['Scraped Text'] = text
                document.close()

                # Adding the file info to the list
                directory_data.append(file_info)

                # Logging the successful extraction
                logging.info(f"Successfully scraped {file_path}")
            except Exception as e:
                # Logging any errors
                logging.error(f"Error scraping {file_path}: {str(e)}")

# Converting the list of dictionaries into a DataFrame
df = pd.DataFrame(directory_data)

In [7]:
df

,File,File Path,Directory Level 1,Scraped Text
0,VEm_01_1.pdf,3Corpus_VEm_dividido\VEm_01_1.pdf,3Corpus_VEm_dividido,junho e julho de 2020\nnúmero 1\nMesmo \nfrent...
1,VEm_01_2.pdf,3Corpus_VEm_dividido\VEm_01_2.pdf,3Corpus_VEm_dividido,em Intercultural Competence. São \nalunos de Q...
2,VEm_01_3.pdf,3Corpus_VEm_dividido\VEm_01_3.pdf,3Corpus_VEm_dividido,"Fatec \nIndaiatuba: \nalém \nde\nNóbrega, part..."
3,VEm_01_4.pdf,3Corpus_VEm_dividido\VEm_01_4.pdf,3Corpus_VEm_dividido,"Fatecs) se distribuíram em seis \ntimes, para ..."
4,VEm_02_1.pdf,3Corpus_VEm_dividido\VEm_02_1.pdf,3Corpus_VEm_dividido,Esta segunda edição de VEm \ncom PCI traz uma ...
...,...,...,...,...
154,VEm_30_05.pdf,3Corpus_VEm_dividido\VEm_30_05.pdf,3Corpus_VEm_dividido,p. 10\nnúmero 30| julho e agosto | 2025\nVirtu...
155,VEm_30_1.pdf,3Corpus_VEm_dividido\VEm_30_1.pdf,3Corpus_VEm_dividido,número 30| julho e agosto | 2025\nVirtual \nEx...
156,VEm_30_2.pdf,3Corpus_VEm_dividido\VEm_30_2.pdf,3Corpus_VEm_dividido,"Osvaldo Succi Junior, coordenador da área de A..."
157,VEm_30_3.pdf,3Corpus_VEm_dividido\VEm_30_3.pdf,3Corpus_VEm_dividido,p. 10\nnúmero 30| julho e agosto | 2025\nVirtu...


## Identify rows where `Scraped Text` is an empty string and drop them

In [8]:
# Identify rows where 'Scraped Text' is an empty string and display them
empty_rows = df[df['Scraped Text'] == '']
empty_rows if not empty_rows.empty else pd.DataFrame(columns=df.columns)

,File,File Path,Directory Level 1,Scraped Text
68,VEm_16_2.pdf,3Corpus_VEm_dividido\VEm_16_2.pdf,3Corpus_VEm_dividido,
69,VEm_16_3.pdf,3Corpus_VEm_dividido\VEm_16_3.pdf,3Corpus_VEm_dividido,
70,VEm_16_4.pdf,3Corpus_VEm_dividido\VEm_16_4.pdf,3Corpus_VEm_dividido,
71,VEm_16_5.pdf,3Corpus_VEm_dividido\VEm_16_5.pdf,3Corpus_VEm_dividido,
73,VEm_17_2.pdf,3Corpus_VEm_dividido\VEm_17_2.pdf,3Corpus_VEm_dividido,
74,VEm_17_3.pdf,3Corpus_VEm_dividido\VEm_17_3.pdf,3Corpus_VEm_dividido,
75,VEm_17_4.pdf,3Corpus_VEm_dividido\VEm_17_4.pdf,3Corpus_VEm_dividido,
76,VEm_17_5.pdf,3Corpus_VEm_dividido\VEm_17_5.pdf,3Corpus_VEm_dividido,
77,VEm_17_6.pdf,3Corpus_VEm_dividido\VEm_17_6.pdf,3Corpus_VEm_dividido,
78,VEm_17_7.pdf,3Corpus_VEm_dividido\VEm_17_7.pdf,3Corpus_VEm_dividido,


In [9]:
# Identify rows where 'Scraped Text' is an empty string and drop them
df = df[df['Scraped Text'] != ''].reset_index(drop=True)
df

,File,File Path,Directory Level 1,Scraped Text
0,VEm_01_1.pdf,3Corpus_VEm_dividido\VEm_01_1.pdf,3Corpus_VEm_dividido,junho e julho de 2020\nnúmero 1\nMesmo \nfrent...
1,VEm_01_2.pdf,3Corpus_VEm_dividido\VEm_01_2.pdf,3Corpus_VEm_dividido,em Intercultural Competence. São \nalunos de Q...
2,VEm_01_3.pdf,3Corpus_VEm_dividido\VEm_01_3.pdf,3Corpus_VEm_dividido,"Fatec \nIndaiatuba: \nalém \nde\nNóbrega, part..."
3,VEm_01_4.pdf,3Corpus_VEm_dividido\VEm_01_4.pdf,3Corpus_VEm_dividido,"Fatecs) se distribuíram em seis \ntimes, para ..."
4,VEm_02_1.pdf,3Corpus_VEm_dividido\VEm_02_1.pdf,3Corpus_VEm_dividido,Esta segunda edição de VEm \ncom PCI traz uma ...
...,...,...,...,...
143,VEm_30_05.pdf,3Corpus_VEm_dividido\VEm_30_05.pdf,3Corpus_VEm_dividido,p. 10\nnúmero 30| julho e agosto | 2025\nVirtu...
144,VEm_30_1.pdf,3Corpus_VEm_dividido\VEm_30_1.pdf,3Corpus_VEm_dividido,número 30| julho e agosto | 2025\nVirtual \nEx...
145,VEm_30_2.pdf,3Corpus_VEm_dividido\VEm_30_2.pdf,3Corpus_VEm_dividido,"Osvaldo Succi Junior, coordenador da área de A..."
146,VEm_30_3.pdf,3Corpus_VEm_dividido\VEm_30_3.pdf,3Corpus_VEm_dividido,p. 10\nnúmero 30| julho e agosto | 2025\nVirtu...


## Drop duplicates

In [10]:
# Drop duplicate rows based on 'Scraped Text' column, keeping the first occurrence
dupe_mask = df.duplicated(subset=['Scraped Text'], keep='first')
dropped_duplicates = df[dupe_mask].copy()
dropped_duplicates if not dropped_duplicates.empty else pd.DataFrame(columns=df.columns)

,File,File Path,Directory Level 1,Scraped Text
83,VEm_19_9.pdf,3Corpus_VEm_dividido\VEm_19_9.pdf,3Corpus_VEm_dividido,Boas \nPráticas\nSustentabilidade \né tema de ...


In [11]:
dropped_duplicates

,File,File Path,Directory Level 1,Scraped Text
83,VEm_19_9.pdf,3Corpus_VEm_dividido\VEm_19_9.pdf,3Corpus_VEm_dividido,Boas \nPráticas\nSustentabilidade \né tema de ...


In [12]:
df = df.drop_duplicates(subset=['Scraped Text']).reset_index(drop=True)
df

,File,File Path,Directory Level 1,Scraped Text
0,VEm_01_1.pdf,3Corpus_VEm_dividido\VEm_01_1.pdf,3Corpus_VEm_dividido,junho e julho de 2020\nnúmero 1\nMesmo \nfrent...
1,VEm_01_2.pdf,3Corpus_VEm_dividido\VEm_01_2.pdf,3Corpus_VEm_dividido,em Intercultural Competence. São \nalunos de Q...
2,VEm_01_3.pdf,3Corpus_VEm_dividido\VEm_01_3.pdf,3Corpus_VEm_dividido,"Fatec \nIndaiatuba: \nalém \nde\nNóbrega, part..."
3,VEm_01_4.pdf,3Corpus_VEm_dividido\VEm_01_4.pdf,3Corpus_VEm_dividido,"Fatecs) se distribuíram em seis \ntimes, para ..."
4,VEm_02_1.pdf,3Corpus_VEm_dividido\VEm_02_1.pdf,3Corpus_VEm_dividido,Esta segunda edição de VEm \ncom PCI traz uma ...
...,...,...,...,...
142,VEm_30_05.pdf,3Corpus_VEm_dividido\VEm_30_05.pdf,3Corpus_VEm_dividido,p. 10\nnúmero 30| julho e agosto | 2025\nVirtu...
143,VEm_30_1.pdf,3Corpus_VEm_dividido\VEm_30_1.pdf,3Corpus_VEm_dividido,número 30| julho e agosto | 2025\nVirtual \nEx...
144,VEm_30_2.pdf,3Corpus_VEm_dividido\VEm_30_2.pdf,3Corpus_VEm_dividido,"Osvaldo Succi Junior, coordenador da área de A..."
145,VEm_30_3.pdf,3Corpus_VEm_dividido\VEm_30_3.pdf,3Corpus_VEm_dividido,p. 10\nnúmero 30| julho e agosto | 2025\nVirtu...


## Rename the columns

In [13]:
df = df.rename(columns={'Directory Level 1': 'Root Directory'})

## Create the column `Text ID`

In [14]:
df['Text ID'] = 't' + df.index.astype(str).str.zfill(6)

In [15]:
df.dtypes

File              object
File Path         object
Root Directory    object
Scraped Text      object
Text ID           object
dtype: object

## Export to a file

In [16]:
df[['Root Directory', 'File', 'File Path', 'Text ID', 'Scraped Text']].to_json(f"{output_directory}/cl_st1_ph1_claudia_scraped.jsonl", orient='records', lines=True)

## Import data into a DataFrame

In [17]:
df = pd.read_json(f'{output_directory}/cl_st1_ph1_claudia_scraped.jsonl', lines=True)

In [18]:
df.dtypes

Root Directory    object
File              object
File Path         object
Text ID           object
Scraped Text      object
dtype: object

In [19]:
df

,Root Directory,File,File Path,Text ID,Scraped Text
0,3Corpus_VEm_dividido,VEm_01_1.pdf,3Corpus_VEm_dividido\VEm_01_1.pdf,t000000,junho e julho de 2020\nnúmero 1\nMesmo \nfrent...
1,3Corpus_VEm_dividido,VEm_01_2.pdf,3Corpus_VEm_dividido\VEm_01_2.pdf,t000001,em Intercultural Competence. São \nalunos de Q...
2,3Corpus_VEm_dividido,VEm_01_3.pdf,3Corpus_VEm_dividido\VEm_01_3.pdf,t000002,"Fatec \nIndaiatuba: \nalém \nde\nNóbrega, part..."
3,3Corpus_VEm_dividido,VEm_01_4.pdf,3Corpus_VEm_dividido\VEm_01_4.pdf,t000003,"Fatecs) se distribuíram em seis \ntimes, para ..."
4,3Corpus_VEm_dividido,VEm_02_1.pdf,3Corpus_VEm_dividido\VEm_02_1.pdf,t000004,Esta segunda edição de VEm \ncom PCI traz uma ...
...,...,...,...,...,...
142,3Corpus_VEm_dividido,VEm_30_05.pdf,3Corpus_VEm_dividido\VEm_30_05.pdf,t000142,p. 10\nnúmero 30| julho e agosto | 2025\nVirtu...
143,3Corpus_VEm_dividido,VEm_30_1.pdf,3Corpus_VEm_dividido\VEm_30_1.pdf,t000143,número 30| julho e agosto | 2025\nVirtual \nEx...
144,3Corpus_VEm_dividido,VEm_30_2.pdf,3Corpus_VEm_dividido\VEm_30_2.pdf,t000144,"Osvaldo Succi Junior, coordenador da área de A..."
145,3Corpus_VEm_dividido,VEm_30_3.pdf,3Corpus_VEm_dividido\VEm_30_3.pdf,t000145,p. 10\nnúmero 30| julho e agosto | 2025\nVirtu...


## Copy each original PDF file to ease the text review process

In [20]:
for index, row in df.iterrows():
    source_path = row['File Path']
    new_name = row['Text ID'] + '.pdf'
    destination_path = os.path.join(output_directory, new_name)
    try:
        shutil.copy(source_path, destination_path)
        logging.info(f"Copied: {source_path} to {destination_path}")
    except Exception as e:
        logging.error(f"Error copying {source_path}: {str(e)}")

## Export each scraped text to individual files for review

This cell has been set to 'raw' in order to avoid unintended overwrite of the the reviewed texts.

## Manual inspection and clean up

Inspect each document in TXT format and:

- Remove titles and subtitles
- Remove headers and footers
- Remove elements such as tables, figures, references and appendices
- **Separate sets of lines that constitute paragraphs with an empty line**

### Example

```
Mesmo
frente
às
diversidades
enfrentadas
nos  dias
de
hoje,
devemos manter o foco em coisas
simples e importantes. Uma delas é
a determinação com a qual a Cesu
vem enfrentando as turbulências e
a
resiliência
de
nosso
corpo
administrativo e docente. Também
é importante celebrar o sucesso da
equipe em lidar com o grande
desafio de oferecer mais e maiores
oportunidades de aprendizagem aos
alunos das Fatecs.

O número inaugural da newsletter
VEm com PCI traz a revolução dos
intercâmbios
virtuais
(VE Virtual Exchange)
tropicalizados para o
ambiente educacional
das Fatecs por meio
dos
Projetos
Colaborativos
Internacionais (PCIs). Esperamos
que vocês participem direta ou
indiretamente de nossos esforços
para levar a internacionalização em
casa às mais diversas localidades do
estado de São Paulo.
```

## Merge lines into paragraphs

In [23]:
# Defining a function to tokenise the paragraphs of each article
def paragraph_tokenise(text):
    lines = text.split('\n')
    paragraphs = []
    paragraph = ''

    for line in lines:
        if line.strip():
            cleaned_line = ' '.join(line.split())  # Remove extra spaces within the line
            paragraph += ' ' + cleaned_line.strip()  # Join subsequent lines into a paragraph
        else:
            paragraphs.append(paragraph.strip())  # If there is an empty line, the paragraph consolidated so far is added to the list of paragraphs
            paragraph = ''  # The paragraph variable is cleared out

    if paragraph:
        paragraphs.append(paragraph.strip())  # The last paragraph is added to the list of paragraphs

    tokenised_paragraphs = '\n'.join(paragraphs)  # The list of paragraphs is compiled into a text with each paragraph as a separate line

    return tokenised_paragraphs

# Defining a function to read the content of a TXT file
def read_txt_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        logging.error(f"Error reading file {file_path}: {e}")
        return None

# Defining a function to save the paragraph-tokenised articles into TXT files
def save_paragraph_tokenised_file(output_text_content, output_file):
    try:
        with open(output_file, 'w', encoding='utf-8') as output_txt_file:
            output_txt_file.write(output_text_content)
        logging.info(f"Successfully saved tokenised file: {output_file}")
    except Exception as e:
        logging.error(f"Error saving file {output_file}: {e}")

# Iterating through each row in the DataFrame and add the text content
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Processing files'):
    text_id = row['Text ID']
    txt_file_path = os.path.join(output_directory, f"{text_id}.txt")
    if os.path.exists(txt_file_path):
        text_content = read_txt_file(txt_file_path)
        if text_content:
            paragraph_tokenised_text_content = paragraph_tokenise(text_content)
            save_paragraph_tokenised_file(paragraph_tokenised_text_content, f"{output_directory}/{text_id}_tokenised.txt")
    else:
        logging.warning(f"File not found: {txt_file_path}")

Processing files: 100%|██████████| 147/147 [00:00<00:00, 1308.64it/s]
